# ------------------------------ TANDEM model ------------------------------
Make sure the machine has more than 24 GB RAM and 8 CPU cores for running this code

This code uses knowledge graph representations of patients (called patient SPOKEsig) which you have to download before running this notebook (as mentioned in the [README instructions](https://github.com/BaranziniLab/TANDEM#instructions)).

In [1]:
from utility import *

2022-10-03 11:30:42.337183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 11:30:42.498430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 11:30:42.498455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-03 11:30:42.524697: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 11:30:44.596127: W tensorflow/stream_executor/pla

# Load model 
### ------------------------------ Note --------------------------------
       train_flag = False --> Use pre-trained model
       train_flag = True --> Train a new model 

In [2]:
temporal_model = load_model_type(model_type="temporal", train_flag=False)
non_temporal_model = load_model_type(model_type="non_temporal", train_flag=False)
tandem_model = load_tandem(temporal_model, non_temporal_model, train_flag=False)


Selected to use pre-trained temporal model
Loading pre-trained temporal model


/pool0/home/karthik/tandem_2/TANDEM/venv/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/pool0/home/karthik/tandem_2/TANDEM/venv/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


Selected to use pre-trained non-temporal model
Loading pre-trained non-temporal model
Selected to use pre-trained TANDEM model
Loading pre-trained TANDEM model


2022-10-03 11:30:52.813920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-03 11:30:52.813968: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-03 11:30:52.814006: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (leo.ucsf.edu): /proc/driver/nvidia/version does not exist
2022-10-03 11:30:52.814281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Compute model predictions

In [3]:
y_score_temporal = get_predictions(temporal_model, temporal_test_data)
print("AUC score of temporal predictions = ", round(get_auc(test_metadata.label.values, y_score_temporal), 2))

y_score_non_temporal = get_predictions(non_temporal_model, non_temporal_test_data)
print("AUC score of non-temporal predictions = ", round(get_auc(test_metadata.label.values, y_score_non_temporal), 2))

y_score_tandem, optThresh = get_tandem_predictions(tandem_model, 
                                                   temporal_model, 
                                                   non_temporal_model, 
                                                   temporal_test_data, 
                                                   non_temporal_test_data, 
                                                   test_metadata)
print("AUC score of TANDEM predictions = ", round(get_auc(test_metadata.label.values, y_score_tandem), 2))

AUC score of temporal predictions =  0.8
AUC score of non-temporal predictions =  0.73
63/63 [==============================] - 0s 1ms/step
AUC score of TANDEM predictions =  0.85


In [4]:
# PD patient example
patient_index = 14
get_tandem_single_patient_prediction(patient_index, 
                                     optThresh, 
                                     temporal_model, 
                                     non_temporal_model, 
                                     tandem_model, 
                                     test_metadata)

1/1 [==============================] - 0s 26ms/step
********** Patient prediction ***********
Label :  PD
TANDEM prediction :  PD
TANDEM prediction score : 0.75


In [5]:
# Non-PD patient example
patient_index = 34
get_tandem_single_patient_prediction(patient_index, 
                                     optThresh, 
                                     temporal_model, 
                                     non_temporal_model, 
                                     tandem_model, 
                                     test_metadata)

1/1 [==============================] - 0s 17ms/step
********** Patient prediction ***********
Label :  Non-PD
TANDEM prediction :  Non-PD
TANDEM prediction score : 0.04
